In [ ]:
#  STEP 1 — Install dependencies
!pip install transformers datasets accelerate evaluate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers -q

In [ ]:
import transformers
transformers.__version__


'4.57.1'

In [ ]:
#  STEP 2 — Import libraries
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
import evaluate
import torch


In [ ]:
#  STEP 3 — Load dataset
train_df = pd.read_csv("/content/train.csv")   # path to Mercor train.csv
test_df  = pd.read_csv("/content/test.csv")    # path to Mercor test.csv


In [ ]:
import pandas as pd


train_df["text"] = train_df["topic"] + " " + train_df["answer"]
test_df["text"]  = test_df["topic"]  + " " + test_df["answer"]



In [ ]:
import pandas as pd


train_df["labels"] = train_df["is_cheating"]
# Convert to HF Dataset
train_ds = Dataset.from_pandas(train_df)

In [ ]:
#  STEP 4 — Tokenize data
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

def preprocess(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

train_ds = train_ds.map(preprocess, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

In [ ]:
#  STEP 5 — Split into train/validation
train_test = train_ds.train_test_split(test_size=0.2, seed=42)
train_split = train_test["train"]
val_split   = train_test["test"]


In [ ]:
#  STEP 6 — Load model
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#  STEP 7 — Define metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
#  STEP 8 — Training arguments

training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=1,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
#  STEP 9 — Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_split,
    eval_dataset=val_split,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-63976143.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


In [ ]:

#  STEP 10 — Evaluate
trainer.evaluate()


In [ ]:
#  STEP 11 — Predict on test data
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(preprocess, batched=True)
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), dim=1).numpy()

submission = pd.DataFrame({
    "id": test_df["id"],
    "is_cheating": preds
})
submission.to_csv("submission_roberta.csv", index=False)
print("✅ submission_roberta.csv saved!")